In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive', force_remount = True) # 此處需要登入google帳號
import os
if not os.path.exists('/content/drive/MyDrive/AICup/yolo_preprocess'):
  os.mkdir('/content/drive/MyDrive/AICup/yolo_preprocess')
os.chdir('/content/drive/MyDrive/AICup/yolo_preprocess')

Mounted at /content/drive


In [ ]:
! pip install numpy
! pip install matplotlib
! pip install opencv-python
! pip install ultralytics
! pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.2/562.2 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 22.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def ensure_dir(path):
  if os.path.exists(path): return
  dir = path.split('/')
  for i in range(1, len(dir)+1 ):
    if not os.path.exists('/'.join(dir[0:i])):
      os.mkdir('/'.join(dir[0:i]))

In [ ]:
target_path = 'out/sorted_player'
resource_path = 'Resource/hitframe'
resource_csv_path = 'Resource/hitframe/hitframe.csv'
ensure_dir(target_path)
ensure_dir(resource_path)

In [ ]:
from math import inf
import cv2
from typing import List
import pandas as pd
from pandas import DataFrame
from tqdm import tqdm
import contextlib
import sys

In [ ]:
from ultralytics import YOLO
from ultralytics.yolo.engine.results import Results

In [ ]:
CFG={
  'iou':0.5,
  'model':"yolov8x.pt",
  'center':(640, 470)
  'out_border': 10
}

In [ ]:
%%capture
def detect(image, model):
    '''
    return two box, which sequently represent (A, B)
    '''
    def dissq(a, b):
        return (a[0]-b[0])**2+(a[1]-b[1])**2
    
    results:List[Results] = model(image, augment=True, verbose=False)
    for result in results:
        people = filter(lambda x: x.cls[0]==0 and x.conf[0]>0.1, result.boxes)
        player1 = (None, inf, None)
        player2 = (None, inf, None)
        for box in people:
            x1, y1, x2, y2 = map(int, box[0].xyxy[0])
            if (dis:=dissq(((x1+x2)/2, y2/2), CFG['center']))<player2[1]:
                player2 = (box, dis, y2)
                if player2[1]<player1[1]:
                    player1, player2 = player2, player1
        if player1[2]>player2[2] : player1, player2 = player2, player1
    return player1[0][0], player2[0][0]

In [ ]:
model = YOLO(CFG['model'])
model.overrides['iou'] = CFG['iou']
data:DataFrame = pd.read_csv(resource_csv_path)

In [ ]:
for i in tqdm(range(1426, data.shape[0])):
    row = data.iloc[i]
    try:
        org = cv2.imread(f'{resource_path}/hitframe_val/{row["VideoName"]}_hitframe_{row["HitFrame"]-1}.png')
        A, B = detect(org, model)
        x1, y1, x2, y2 = map(int, A.xyxy[0])
        x1, y1, x2, y2 = x1-=CFG['out_border'], y1-=CFG['out_border'], x2+=CFG['out_border'], y2+=CFG['out_border']
        imgA = org[y1:y2, x1:x2]
        group, hit_frame = row["VideoName"].replace(".", ""), row["HitFrame"]-1
        fileNameA = f'{group}_A_hitframe_{hit_frame}_x{x1}_y{y1}.png'
        x1, y1, x2, y2 = map(int, B.xyxy[0])
        x1, y1, x2, y2 = x1-=CFG['out_border'], y1-=CFG['out_border'], x2+=CFG['out_border'], y2+=CFG['out_border']
        imgB = org[y1:y2, x1:x2]
        fileNameB = f'{group}_B_hitframe_{hit_frame}_x{x1}_y{y1}.png'
        p1, p2 = (imgA, fileNameA), (imgB, fileNameB)
        cv2.imwrite(f'{target_path}/{p1[1]}', p1[0])
        cv2.imwrite(f'{target_path}/{p2[1]}', p2[0])
    except Exception as e:
        print(e)

100%|██████████| 130/130 [02:29<00:00,  1.15s/it]


In [ ]:
data.shape[0]

1556

In [ ]:
len(os.listdir(target_path))

3112